<a href="https://colab.research.google.com/github/LSSTDESC/DifferentiableHOS/blob/main/notebooks/spectrum_for_cosmology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pylab inline 
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from flowpm.tfbackground import cosmo,z2a,a2z,rad_comoving_distance, a_of_chi
from flowpm.tfpower import linear_matter_power
from flowpm.raytracing import  lightcone, Born
import flowpm 
from flowpm.scipy.interpolate import interp_tf

Populating the interactive namespace from numpy and matplotlib


In [2]:
nc=[32,32,320]   # size of the cube, number of cells
plane_size=32                    # number of pixel for x and  y 
Boxsize=[200,200,2000]          # Physical size of the cube
field=5.
n_steps=2

In [3]:
import tensorflow as tf
import numpy as np

def measure_power_spectrum_tf(map_data,field,nc_xy):
    """
    Measures power spectrum or 2d data
    
    Parameters:
    -----------
    map_data: map (n x n)
    
    field: int or float
        transveres degres of the field
        
    nc_xy : int
           Number of pixel for x and  y 
          
    Returns
    -------
    ell: tf.TensorArray
    power spectrum: tf.TensorArray
    """
    
    def radial_profile_tf(data):
        """
        Compute the radial profile of 2d image
        
        Parameters:
        -----------
        data: 2d image
        
        Returns
        -------
        radial profile
        """
        center = data.shape[0]/2
        y, x = np.indices((data.shape))
        r = tf.math.sqrt((x - center)**2 + (y - center)**2)
        r=tf.cast(r,dtype=tf.int32)
        tbin=tf.math.bincount(tf.reshape(r,[-1]), tf.reshape(data,[-1]))
        nr = tf.math.bincount(tf.reshape(r,[-1]))
        radialprofile=tf.cast(tbin,dtype=tf.float32)/tf.cast(nr,dtype=tf.float32)
        return radialprofile
    
    
    def resolution(field,nc_xy):
        """
        pixel resolution

        Returns
        -------
          float
         pixel resolution
         
        """
        return  field*60/nc_xy
    
    def pixel_size_tf(field,nc_xy):
        """
        pixel size

        Returns
        -------
        
        pizel size: float
        pixel size
        
        Notes
        -----
    
        The pixels size is given by:
    
        .. math::
    
            pixel_size =  =pi * resolution / 180. / 60. #rad/pixel
         
        """
        return field/nc_xy / 180 *np.pi 
    data_ft = tf.signal.fftshift(tf.signal.fft2d(map_data)) / map_data.shape[0]
    nyquist = tf.cast(map_data.shape[0]/2,dtype=tf.int32)
    power_spectrum = radial_profile_tf(tf.math.real(data_ft*tf.math.conj(data_ft)))[:nyquist]
    power_spectrum = power_spectrum*pixel_size_tf(field,nc_xy)**2
    return power_spectrum

In [4]:
@tf.function
def power_spectrum_for_cosmology(
              Omega0_m,
              sigma8):
    cosmology=cosmo.copy()
    cosmology['Omega0_m']=tf.convert_to_tensor(Omega0_m,dtype=tf.float32)
    cosmology['sigma8']=tf.convert_to_tensor(sigma8,dtype=tf.float32)
    a_s=z2a(tf.convert_to_tensor(1.00, dtype=tf.float32))
    r = tf.linspace(0,2000,2)
    a=a_of_chi(cosmology,r)
    ds=rad_comoving_distance(cosmology,a_s)
    init_stages = tf.linspace(0.1, a[-1], 2)
    
    # so, instead of returning the linear matter power spectrum to be computed on the fly
    # we can precompute it and return an interpolation function
    k = tf.constant(np.logspace(-4, 1, 256), dtype=tf.float32)
    pk = linear_matter_power(cosmology, k)
    pk_fun = lambda x: tf.cast(tf.reshape(interp_tf(tf.reshape(tf.cast(x, tf.float32), [-1]), k, pk), x.shape), tf.complex64)
    
    initial_conditions = flowpm.linear_field(nc,    
                                            Boxsize, 
                                             pk_fun,         
                                             batch_size=1)
    # Sample particles
    state = flowpm.lpt_init(initial_conditions, 0.1)   
    # Evolve particles down to z=0
    final_state = flowpm.nbody(state, init_stages, nc)         
    # Retrieve final density field
    state, lps_a, lps=lightcone(final_state, a[::-1], 
                                nc, 
                                field*60/plane_size, plane_size,
                                cosmology)
    k_map=Born(lps_a,lps,ds,nc,Boxsize,plane_size,field,cosmology)
    k_map=tf.cast(k_map,dtype=tf.complex64)
    power_spectrum=measure_power_spectrum_tf(k_map,field,plane_size)
    return power_spectrum, k_map

In [5]:
@tf.function
def compute_jacobian(Omega0_m, sigma8):
    Omega0_m=tf.convert_to_tensor(Omega0_m,dtype=tf.float32)
    sigma8=tf.convert_to_tensor(sigma8,dtype=tf.float32)
    
    params = tf.stack([Omega0_m, sigma8])
    with tf.GradientTape() as tape:
        tape.watch(params)
        power_spectrum, kmap= power_spectrum_for_cosmology(
              params[0], params[1])
        # And we are actually going to try to compress the PS 
        # to avoid needing too much memory
        power_spectrum = power_spectrum[::2] # it should be of size 8
    return tape.jacobian(power_spectrum, params,
                         experimental_use_pfor=False)

In [6]:
jacobian = compute_jacobian(0.3075,0.8159)

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


In [7]:
jacobian

<tf.Tensor: shape=(8, 2), dtype=float32, numpy=
array([[ 5.7298254e-04,  4.9726241e-09],
       [ 1.1478999e-07,  4.2208161e-08],
       [ 3.6076607e-08,  1.5743144e-08],
       [ 1.9482027e-08,  7.7278184e-09],
       [ 7.7816518e-09,  5.2178133e-09],
       [ 6.0456449e-09,  1.7883484e-09],
       [ 4.6106026e-09,  1.1450587e-09],
       [ 3.2822087e-09, -2.3425301e-10]], dtype=float32)>